In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("ggplot")

%matplotlib inline

## Split-Apply-Combine

One powerful paradigm for analyzing data is the "Split-Apply-Combine" strategy

This strategy has three steps:

1. `Split`: split the data into groups based on values
2. `Apply`: apply a function or routine to each group separately
3. `Combine`: combine the output of the apply step into a DataFrame, using the group identifiers as the index

We will cover the main components in this lecture, but we encourage you to also study the [official documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html) to learn more about what is possible

To describe the concepts, we will need some data

We will begin with a simple made-up dataset to discuss the concepts and then work through extended example and exercises with real data

In [47]:
bball = pd.DataFrame({
    "Year": [2015, 2016, 2017]*2,
    "Player": ["Curry"]*3 + ["Durant"]*3,
    "Team": ["GSW"]*3 + ["OKC"] + ["GSW"]*2,
    "Pts": [30.1, 25.3, 27.1, 28.2, 25.1, 25.9],
    "Assist": [6.7, 6.6, 6.4, 5.0, 4.8, 5.3],
    "Rebound": [4.5, 3.9, 3.4, 4.4, 6.5, 7.6]
})

bball.info()
bball

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
Assist     6 non-null float64
Player     6 non-null object
Pts        6 non-null float64
Rebound    6 non-null float64
Team       6 non-null object
Year       6 non-null int64
dtypes: float64(3), int64(1), object(2)
memory usage: 368.0+ bytes


,Assist,Player,Pts,Rebound,Team,Year
0,6.7,Curry,30.1,4.5,GSW,2015
1,6.6,Curry,25.3,3.9,GSW,2016
2,6.4,Curry,27.1,3.4,GSW,2017
3,5.0,Durant,28.2,4.4,OKC,2015
4,4.8,Durant,25.1,6.5,GSW,2016
5,5.3,Durant,25.9,7.6,GSW,2017


### Simple example

To perform the **_SPLIT_** step, we call the `groupby` method on our DataFrame

The first argument to `groupby` is a description of how we want to construct groups

In the most basic version we will pass a string identifying the column name

In [31]:
gbA = bball.groupby("Team")

The `type` of variable we get back is a `DataFrameGroupBy`, which we will sometimes refer to as GroupBy for short. The command below shows the groups that we have access to.

In [32]:
gbA.groups

{'GSW': Int64Index([0, 1, 2, 4, 5], dtype='int64'),
 'OKC': Int64Index([3], dtype='int64')}

Then this is a way to grab one of the groups...

In [35]:
gbA.get_group("OKC")

,Assist,Player,Pts,Rebound,Team,Year
3,5.0,Durant,28.2,4.4,OKC,2015


We can **_APPLY_** some of our favorite reduction functions directly on the `GroupBy` object

**Exercise**: With your neighbor, look closely at the output of the cells below

How did pandas compute the sum of `gbA`? What happened to the `NaN` entries in column `C`?

Hint: try `gbA.count()` or `gbA.mean()` if you can't decide what happened to the `NaN`s

In [37]:
gbA.mean()

,Assist,Pts,Rebound,Year
Team,,,,
GSW,5.96,26.7,5.18,2016.2
OKC,5.00,28.2,4.40,2015.0


**Exercise**: Use (tab completion) to see what other reductions are defined for GroupBy objects

Pick three and evaluate them in the cells below

Does the output of each of these commands have the same features as the output of `gbA.sum()` from above? If not, what is different?

You can also apply on only one column, e.g. we just want the mean of B, by group.

In [39]:
gbA["Pts"].mean()

Team
GSW    26.7
OKC    28.2
Name: Pts, dtype: float64

One final thing, we can apply an aggregation by column using a dictionary. 

In [53]:
transform_dict = {"Pts": "sum", "Year": "first"}


gbA.agg(transform_dict)

,Year,Pts
Team,,
GSW,2015,133.5
OKC,2015,28.2


In [64]:
gbA.apply(pd.groupby.first).Pts

AttributeError: 'function' object has no attribute 'first'

If we pass a list of strings to `groupby`, it will group based on unique combinations of values from all columns in the list

Let's see an example

In [42]:
gbAB = bball.groupby(["Team", "Player"])
type(gbAB)

pandas.core.groupby.DataFrameGroupBy

In [43]:
gbAB.groups

{('GSW', 'Curry'): Int64Index([0, 1, 2], dtype='int64'),
 ('GSW', 'Durant'): Int64Index([4, 5], dtype='int64'),
 ('OKC', 'Durant'): Int64Index([3], dtype='int64')}

In [50]:
gbAB.get_group((1,1)).Pts.mean()

gbAB.get_group(("OKC","Durant")).Pts.mean()

KeyError: (1, 1)

### "Not reductions": the `apply` method

We can apply "not reductions" to DataFrames. Here is the idea, is we want to grab a subset of the series by group. In the basketball case, lets get each players best two years, and see what the characteristics are.  We can do the same with GroupBy objects using the `.apply` method

Let's see an example

In [65]:
def best_by_pts(df):
    return df.nlargest(2, "Pts")

In [70]:
gbA = bball.groupby("Player")

new_df = gbA.apply(best_by_pts)

Interesting, for both players, 2016 was the relativly weak year. Do you see what is going on here?

**Exercise**: This exercise has a few steps:

1. Write a function that, given a DataFrame, creates a new column that computes each players points as a deviation from career mean, and then the function returns the entire DataFrame
2. Apply the function to `bball`
3. With your neighbor describe what happened?

In [78]:
def mean_dev_by_pts(df):
    df["mean_dev"] = df["Pts"] - df["Pts"].mean()
    return df

In [79]:
gbA = bball.groupby("Player")

gbA.apply(mean_dev_by_pts)

,Assist,Player,Pts,Rebound,Team,Year,mean_dev
0,6.7,Curry,30.1,4.5,GSW,2015,2.6
1,6.6,Curry,25.3,3.9,GSW,2016,-2.2
2,6.4,Curry,27.1,3.4,GSW,2017,-0.4
3,5.0,Durant,28.2,4.4,OKC,2015,1.8
4,4.8,Durant,25.1,6.5,GSW,2016,-1.3
5,5.3,Durant,25.9,7.6,GSW,2017,-0.5


In [128]:
pgame = pd.read_csv("C:\\data_bootcamp\\data_bootcamp_spring_2018\\data\\2016-17_playerBoxScore.csv")

pgame["gmDate"] = pd.to_datetime(pgame["gmDate"], format="%Y-%m-%d")

In [129]:
durant_game = pgame[pgame.playLNm == "Durant"]

In [132]:
durant_game.groupby(pd.Grouper(key="gmDate", freq="M")).count()

,gmTime,seasTyp,playLNm,playFNm,teamAbbr,teamConf,teamDiv,teamLoc,teamRslt,teamDayOff,...,playFT%,playORB,playDRB,playTRB,opptAbbr,opptConf,opptDiv,opptLoc,opptRslt,opptDayOff
gmDate,,,,,,,,,,,,,,,,,,,,,
2016-10-31,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2016-11-30,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
2016-12-31,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16
2017-01-31,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,14,14,14,14,14,14
2017-02-28,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
2017-03-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-04-30,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
